# Pattern Recognition, Homework 2

* **Author** Haonan Chen
* **Date** 2016/10/22
* **Language** [Julia](http://julialang.org/)

## Read training and testing data

Using `Pkg.add(package)` to install packages.

In [65]:
using DataFrames
using DataFramesMeta
using Distributions

In [ ]:
df_train = readtable("dataset3.txt", separator = '\t')
df_test = readtable("dataset4.txt", separator = '\t')
names!(df_train, cat(1, [Symbol("F$i") for i in 1:10],[Symbol("Sex")]))  # assign a name to each column
names!(df_test,  cat(1, [Symbol("F$i") for i in 1:10],[Symbol("Sex")]))
df_test[df_test[:,:Sex].=="f",:Sex]="F"
head(df_train)

In [ ]:
describe(df_train)

## Sex classification using 2 features

Using `:F3` and `:F5` to do classification.

### Bayes classifier with minimum error

Following procedure:

* Fit Normal Distribution in each class in the training dataa to get parameters $\mu_i$ and $\Sigma_i$.
* Decision boundary can be determined by
$$
    -\frac12[(x-\mu_i)^\mathbf{T}\Sigma_i^{-1}(x-\mu_i)-(x-\mu_j)^\mathbf{T}\Sigma_j^{-1}(x-\mu_j)]-\frac12\ln \frac{|\Sigma_i|}{|\Sigma_j|}+\ln\frac{P(\omega_i)}{P(\omega_j)}=0
$$
* Then we can make prediction on test set.

Note that we are assuming the priori probability $P(\omega_i)=P(\omega_j)=0.5$, so the last item in the formular above can be ignored.

In [119]:
# Distribution for female (female first)
df_train_F = df_train[df_train[:,:Sex].=="F",[:F3,:F5]]
d = fit(MvNormal, Array(df_train_F)')

FullNormal(
dim: 2
μ: [51.7457,163.632]
Σ: [39.2591 16.5807; 16.5807 30.053]
)


In [122]:
# Distribution for male (male first)
df_train_F = df_train[df_train[:,:Sex].=="F",[:F3,:F5]]
d = fit(MvNormal, Array(df_train_F)')

2×2 Array{Float64,2}:
 39.2591  16.5807
 16.5807  30.053 

In [93]:
Σ = ones(3,3)

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [111]:
fit(MvNormal, Array(df_train_F)')

FullNormal(
dim: 2
μ: [51.7457,163.632]
Σ: [39.2591 16.5807; 16.5807 30.053]
)


In [117]:
df_train[:,1:10]

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
1,240.25,16.81,41.0,17.07,155.0,170.0,51.0,162.71,165.05,160.38
2,249.64,17.64,42.0,16.82,158.0,176.0,60.0,160.3,161.27,168.75
3,265.69,30.25,55.0,20.7,163.0,172.0,64.0,184.24,169.06,173.31
4,252.81,21.16,46.0,18.2,159.0,160.0,72.0,182.67,174.44,153.37
5,289.0,25.0,50.0,17.3,170.0,167.0,70.0,198.68,170.79,176.13
6,295.84,24.01,49.0,16.56,172.0,180.0,68.0,175.48,168.16,184.83
7,278.89,26.01,51.0,18.29,167.0,155.0,65.0,170.13,167.75,154.12
8,249.64,19.36,44.0,17.63,158.0,171.0,60.0,162.38,167.69,198.73
9,289.0,28.09,53.0,18.34,170.0,166.0,51.0,170.74,165.71,172.34
10,285.61,39.69,63.0,22.06,169.0,158.0,50.0,183.8,161.8,194.4
